In [78]:
%pip install -U -q langchain_huggingface
%pip install -U -q transformers
%pip install -U -q accelerate
%pip install -U -q langchain_community
%pip install -U -q umap umap-learn
%pip install -U -q langchain_huggingface
%pip install -U -q matplotlib
%pip install -U -q pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from numpy import mean, var, square, mean, ndarray, vstack, concatenate
from matplotlib.pyplot import figure, title, imshow, colorbar, xlabel, ylabel, tight_layout, show, subplots, tight_layout, show, colorbar, scatter, annotate, grid, cm, legend
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pandas import read_parquet
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from torch import bfloat16
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [68]:
splits = {
  'test': 'en-af/test-00000-of-00001.parquet',
  'train': 'en-af/train-00000-of-00001.parquet',
  'validation': 'en-af/validation-00000-of-00001.parquet'
}

In [69]:
documents = [
  "The Porsche 911 Turbo S is a pinnacle of automotive engineering, combining blistering performance with timeless design. Launched as part of the iconic 911 lineup, the Turbo S represents the zenith of Porsche's commitment to performance and luxury. With its distinctive silhouette and aggressive stance, the Turbo S commands attention on the road. Under the hood lies a powerful twin-turbocharged flat-six engine, delivering an exhilarating 640 horsepower and 590 lb-ft of torque.",
  "Acceleration is nothing short of breathtaking; the Turbo S can sprint from 0 to 60 mph in just 2.6 seconds. This remarkable feat is supported by Porsche's advanced all-wheel-drive system, which ensures optimal grip and stability in various driving conditions. The 8-speed dual-clutch transmission (PDK) enhances the driving experience, offering lightning-fast gear shifts that keep the driver engaged.",
  "Beyond its raw power, the Turbo S boasts a sophisticated suspension system that balances comfort and performance. The Porsche Active Suspension Management (PASM) system adjusts the dampers in real-time, providing a smooth ride while maintaining agility on the track. Inside, the cabin is a blend of luxury and sportiness, featuring premium materials, ergonomic design, and cutting-edge technology.",
  "Drivers are treated to an intuitive infotainment system with a high-resolution touchscreen, navigation, and smartphone integration. Safety features are abundant, including adaptive cruise control, lane-keeping assist, and a surround-view camera system. The Turbo S is also customizable, allowing buyers to select from a range of colors, materials, and performance options to suit their tastes.",
  "In terms of design, the Turbo S incorporates aerodynamic elements that enhance both performance and aesthetics. The rear spoiler and active cooling ducts not only improve downforce but also give the car a striking appearance. The Turbo S is a true testament to Porsche's engineering prowess, achieving a perfect balance between speed and elegance.",
  "Driving a Porsche 911 Turbo S is an experience that transcends mere transportation; it is an emotional journey. The sound of the engine, the precision of the steering, and the responsiveness of the brakes create a symphony of sensations that thrill the driver. Whether on a winding mountain road or a racetrack, the Turbo S delivers an unparalleled driving experience.",
  "In conclusion, the Porsche 911 Turbo S stands as a symbol of performance, luxury, and innovation. It embodies the spirit of the brand, appealing to enthusiasts and collectors alike. With its blend of cutting-edge technology and classic design, the Turbo S is not just a car; it is a legacy.",
]

In [70]:
bert_base_nli_mean_tokens_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [71]:
def generate_embeddings(model, documents: list) -> ndarray:
  """
  Generate embeddings for a list of documents using the specified model.
  """
  return model.encode(documents)

In [72]:
embeddings_bert_base_nli_mean_tokens_model = generate_embeddings(bert_base_nli_mean_tokens_model, documents)

In [73]:
text_generation_model_name = "HuggingFaceH4/zephyr-7b-beta"

In [74]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=bfloat16
)

In [77]:
model = AutoModelForCausalLM.from_pretrained(text_generation_model_name)

Loading checkpoint shards: 100%|██████████| 8/8 [00:55<00:00,  6.91s/it]
